In [1]:
import os
%pwd
os.chdir("../")
%pwd


'd:\\Data Science\\END to END Proj\\BloodGroupClassifierBasedonFingerPrint'

In [2]:
## ENTITY
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    input_shape: tuple
    epochs: int
    batch_size: int
    validation_split: float
    learning_rate: float

In [8]:
from src.BloodGroupClassifier.constant import *
from src.BloodGroupClassifier.utils.common import read_yaml,create_directories 
from BloodGroupClassifier.entity.config_entity import DataPreprocessingConfig

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        params = self.params
    
        return DataPreprocessingConfig(
        data_path=Path("artifacts/data_ingestion/"),  # Hardcoded to match original
        img_height=params.IMG_HEIGHT,
        img_width=params.IMG_WIDTH,
        batch_size=params.BATCH_SIZE,
        validation_split=params.VALIDATION_SPLIT
        )
    
    

    def get_model_training_config(self) -> ModelTrainingConfig:
        training_config = self.config.model_training
        params = self.params.model_training
    
        create_directories([training_config.root_dir])

        return ModelTrainingConfig(
        root_dir=Path(training_config.root_dir),
        trained_model_path=Path(training_config.trained_model_path),
        input_shape=tuple(params.input_shape),
        epochs=params.epochs,
        batch_size=params.batch_size,
        validation_split=params.validation_split,
        learning_rate=params.learning_rate
        )

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from pathlib import Path

class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def _build_model(self, num_classes: int) -> tf.keras.Model:
        """Builds the exact model architecture from original code"""
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', 
                         input_shape=self.config.input_shape),
            layers.MaxPooling2D(2, 2),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D(2, 2),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D(2, 2),
            layers.Conv2D(256, (3, 3), activation='relu'),
            layers.MaxPooling2D(2, 2),
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation='softmax')
        ])

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def train(self, train_generator, validation_generator):
        """Trains model with the same parameters as original code"""
        # Check if model already exists
        if Path(self.config.trained_model_path).exists():
            print(f"Model already exists at {self.config.trained_model_path}")
            return tf.keras.models.load_model(self.config.trained_model_path)

        # Build model
        model = self._build_model(num_classes=train_generator.num_classes)
        
        # Train with same parameters as original
        history = model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=self.config.epochs,
            steps_per_epoch=train_generator.samples // self.config.batch_size,
            validation_steps=validation_generator.samples // self.config.batch_size
        )

        # Save in Keras format (SavedModel)
        model.save(self.config.trained_model_path)
        print(f"Model saved to {self.config.trained_model_path}")
        
        return history

In [15]:
from BloodGroupClassifier.components.Data_Preprocessing import DataPreprocessing
try:
    config = ConfigurationManager()
    
    # Get data generators (from preprocessing step)
    preprocessing_config = config.get_data_preprocessing_config()
    data_preprocessor = DataPreprocessing(config=preprocessing_config)
    train_gen, val_gen = data_preprocessor.get_data_generators()
    
    # Get model training config
    training_config = config.get_model_training_config()
    
    # Train model
    trainer = ModelTrainer(config=training_config)
    history = trainer.train(train_gen, val_gen)
    
    print("Training completed successfully!")
    print(f"Final model saved to: {training_config.trained_model_path}")

except Exception as e:
    print(f"Error during model training: {str(e)}")
    raise e

[2025-07-29 22:26:48,958: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-29 22:26:48,958: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-29 22:26:48,976: INFO: common: created directory at: artifacts]


Found 4803 images belonging to 8 classes.
Found 1197 images belonging to 8 classes.
[2025-07-29 22:26:49,680: INFO: Data_Preprocessing: Created generators with:]
[2025-07-29 22:26:49,680: INFO: Data_Preprocessing:   - Image size: 64x64]
[2025-07-29 22:26:49,680: INFO: Data_Preprocessing:   - Batch size: 32]
[2025-07-29 22:26:49,680: INFO: Data_Preprocessing:   - Validation split: 0.2]
[2025-07-29 22:26:49,696: INFO: common: created directory at: artifacts/model_training]
Epoch 1/100
150/150 [==============================] - 143s 927ms/step - loss: 1.3239 - accuracy: 0.4879 - val_loss: 0.7523 - val_accuracy: 0.7171
Epoch 2/100
150/150 [==============================] - 57s 377ms/step - loss: 0.6016 - accuracy: 0.7774 - val_loss: 0.4872 - val_accuracy: 0.8243
Epoch 3/100
150/150 [==============================] - 59s 395ms/step - loss: 0.5073 - accuracy: 0.8080 - val_loss: 0.5514 - val_accuracy: 0.7872
Epoch 4/100
150/150 [==============================] - 55s 367ms/step - loss: 0.4493 